In [ ]:
! pip install contractions

In [1]:
# importing the libraries
import pandas as pd
import numpy as np
import os
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import contractions
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import torch
import torch.nn as nn
import torch.optim as optim


import matplotlib.pyplot as pltA
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time

from collections import Counter, OrderedDict
from itertools import chain
import torch
from tqdm import tqdm
import warnings

# import torchtext
# from torchtext.legacy.datasets import Multi30k
# from torchtext.legacy.data import Field, BucketIterator

# import torchvision
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# import matplotlib.pyplot as plt
# from random import *
# import warnings
# warnings.filterwarnings("ignore")
# from torchvision import datasets
# from torch import Tensor
# import random 
# from random import shuffle

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dheer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [84]:
train_data=pd.read_csv('data/train.csv')
test_data=pd.read_csv('data/test.csv')
validation_data=pd.read_csv('data/validation.csv')

### Training Data

In [85]:
train_data.head()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [86]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287113 entries, 0 to 287112
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          287113 non-null  object
 1   article     287113 non-null  object
 2   highlights  287113 non-null  object
dtypes: object(3)
memory usage: 6.6+ MB


In [87]:
train_data[train_data[['article','highlights']].duplicated()].shape[0]

3098

In [88]:
train_data.drop_duplicates(subset=['article','highlights'],inplace=True)
train_data.shape

(284015, 3)

In [89]:
train_data

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...
...,...,...,...
287108,fffdfb56fdf1a12d364562cc2b9b1d4de7481dee,By . James Rush . Former first daughter Chelse...,Chelsea Clinton said question of running for o...
287109,fffeecb8690b85de8c3faed80adbc7a978f9ae2a,An apologetic Vanilla Ice has given his first ...,"Vanilla Ice, 47 - real name Robert Van Winkle ..."
287110,ffff5231e4c71544bc6c97015cdb16c60e42b3f4,America's most lethal sniper claimed he wished...,America's most lethal sniper made comment in i...
287111,ffff924b14a8d82058b6c1c5368ff1113c1632af,"By . Sara Malm . PUBLISHED: . 12:19 EST, 8 Mar...",A swarm of more than one million has crossed b...


In [90]:
train_data.rename(columns={'highlights':'summary'},inplace=True)

In [91]:
print("Text example:",train_data.iloc[1,1])
print("Summarized samples: ",train_data.iloc[1,2])

Text example: (CNN) -- Ralph Mata was an internal affairs lieutenant for the Miami-Dade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, authorities allege that the 45-year-old longtime officer worked with a drug trafficking organization to help plan a murder plot and get guns. A criminal complaint unsealed in U.S. District Court in New Jersey Tuesday accuses Mata, also known as "The Milk Man," of using his role as a police officer to help the drug trafficking organization in exchange for money and gifts, including a Rolex watch. In one instance, the complaint alleges, Mata arranged to pay two assassins to kill rival drug dealers. The killers would pose as cops, pulling over their targets before shooting them, according to the complaint. "Ultimately, the (organization) decided not to move forward with the murder plot, but Mata still received a payment for setting up the meetings," federal prosecutors said in a statement

In [92]:
def contraction(text):
    return contractions.fix(text)

In [93]:
train_data['article']=train_data['article'].apply(contraction)
train_data['summary']=train_data['summary'].apply(contraction)

In [94]:
train_data.head()

,id,article,summary
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [95]:
import re
import string
punctuation = string.punctuation
punctuation = punctuation.replace('.','')

In [97]:
train=train_data.copy()


In [219]:
train_data=train.copy()

In [220]:
def text_cleaning(text):
    res = re.sub('  +', ' ', text)
    res = re.sub(r'[^\w\s]', ' ', text)
    res = re.sub(r'^https?:\/\/.*[\r\n]*','', text, flags=re.MULTILINE)
    res=re.sub("[^A-Za-z0-9]+",' ',text)
    stop_words = set(stopwords.words('english'))
    res=res.lower()
    res=re.sub(r'\b\w{1,3}\b', '',res)
    word_tokens = word_tokenize(res)
    res = [w for w in word_tokens if w not in stop_words]
    result=' '.join(res)
    return result

In [221]:
train_data['article']=train_data['article'].apply(text_cleaning)
train_data['summary']=train_data['summary'].apply(text_cleaning)

In [231]:
# train_sample.to_csv("Cleaned_Data.csv",index=False)

### Building the model

In [206]:
!pip install -U torchtext==0.10.0

  Using cached torch-1.9.0-cp38-cp38-win_amd64.whl (222.0 MB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\dheer\\AppData\\Local\\Temp\\pip-uninstall-3fozy52b\\lib\\asmjit.dll'
Consider using the `--user` option or check the permissions.



  Attempting uninstall: torch
    Found existing installation: torch 1.10.0
    Uninstalling torch-1.10.0:
      Successfully uninstalled torch-1.10.0


In [248]:
pip install sacremoses


  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895254 sha256=cb415506b354027ba725c48f7373ac0567d667877f224e674c8ed9e20ebe81b7
  Stored in directory: c:\users\dheer\appdata\local\pip\cache\wheels\82\ab\9b\c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4


Successfully built sacremoses


In [207]:
pip install intel-openmp

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
train_data=pd.read_csv('Cleaned_Data.csv')

In [186]:
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True,) # Whether the model returns all hidden-states.

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [187]:
def transformer_prediction(token_tensor,segement_tensor):
    model.eval()
    with torch.no_grad():
        outputs = model(token_tensor, segments_tensors)
        hidden_states = outputs[2]

    token_vecs = hidden_states[-2][0]
    return hidden_states
    

In [188]:
def word_embedding(token_embeddings):
    token_vecs_sum = []
    for token in token_embeddings:
        sum_vec = torch.sum(token[-4:], dim=0)
        sum_vec=torch.reshape(sum_vec,shape=(1,-1))
        token_vecs_sum.append(sum_vec)
    token_vecs_sum=torch.cat(token_vecs_sum,dim=0)
    return token_vecs_sum